#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
Laptime_Schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("driverId", IntegerType(), True),
                                  StructField("lap", IntegerType(), True),
                                  StructField("position", IntegerType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("milliseconds", StringType(), True)])

In [0]:
Laptime_df = spark.read\
    .format("csv")\
    .schema(Laptime_Schema)\
    .option("header", True)\
    .load(f"{raw_folder_path}/{v_file_date}/lap_times")

In [0]:
lap_time_final_df = Laptime_df.withColumnRenamed("raceId", "race_id")\
                            .withColumnRenamed("driverId", "driver_id")\
                            .withColumn("ingestion_date", current_timestamp())\
                            .withColumn("data_source", lit(v_data_source))\
                            .withColumn("file_date", lit(v_file_date))

In [0]:
#for race_id_list in lap_time_final_df.select("race_id").distinct().collect():
#  if (spark._jsparkSession.catalog().tableExists("f1_processed.lap_times")):
#    spark.sql(f"ALTER TABLE f1_processed.lap_times DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")
#lap_time_final_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.lap_times")


##### incremental Load for Delta files

In [0]:
merge_condition = "tgt.lap = src.lap AND tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id"
merge_delta_data(lap_time_final_df, "f1_processed", "lap_times", processed_folder_path, merge_condition, "race_id")

In [0]:
dbutils.notebook.exit("Success")